# Technical Support – SLA Email Automation (Vertex AI + Pub/Sub + Cloud Functions)

This single notebook covers:
- **Step 0 (One-time setup)**: enable APIs, create **Pub/Sub** topic, deploy **Cloud Function** (from ZIP)  
- **Step 1–5 (Per-run in notebook)**: compute SLA results (or reuse your DataFrame) and **publish** events to Pub/Sub so the function sends **Apology/Success** emails.

> Works with your existing project files (e.g., `complaints_train.csv`, `customers.csv`).

## Step 0 — One-time setup in your GCP project

Run these **once** in Cloud Shell or a local terminal authenticated to your GCP project. You can also run them here using `!` if your environment has the `gcloud` CLI.

### 0.1 Enable required APIs
```bash
gcloud services enable   cloudfunctions.googleapis.com   pubsub.googleapis.com   storage.googleapis.com   aiplatform.googleapis.com   secretmanager.googleapis.com
```

### 0.2 Create Pub/Sub topic
```bash
gcloud pubsub topics create sla-email-events
```

### 0.3 Deploy the Cloud Function (Pub/Sub → Email)
You already have the ZIP (or use Console > Cloud Functions > Deploy from ZIP).  
If deploying via CLI and you have the ZIP locally, first unzip or pass the folder path.

```bash
# Example: deploy from a local folder "send_sla_email/"
gcloud functions deploy send_sla_email   --runtime python311   --trigger-topic=sla-email-events   --entry-point=sla_email_trigger   --region=us-central1   --set-env-vars="EMAIL_SMTP_HOST=smtp.gmail.com,EMAIL_SMTP_PORT=587,EMAIL_SMTP_USER=<your_user>,EMAIL_SMTP_PASS=<your_pass>,EMAIL_FROM=support@yourdomain.com"
```

**(Recommended)** Store SMTP creds in Secret Manager and bind via `--set-secrets`:
```bash
gcloud secrets create smtp-user --data-file=<(echo -n '<your_user>')
gcloud secrets create smtp-pass --data-file=<(echo -n '<your_pass>')

gcloud functions deploy send_sla_email   --runtime python311   --trigger-topic=sla-email-events   --entry-point=sla_email_trigger   --region=us-central1   --set-env-vars="EMAIL_SMTP_HOST=smtp.gmail.com,EMAIL_SMTP_PORT=587,EMAIL_FROM=support@yourdomain.com"   --set-secrets="EMAIL_SMTP_USER=smtp-user:latest,EMAIL_SMTP_PASS=smtp-pass:latest"
```

> After this step, the function will listen to the topic **sla-email-events** and will send **Apology** when `sla_breached=True` or **Success** otherwise.

## Step 1 — Install Pub/Sub client (only once per environment)

In [ ]:
# If needed
%pip install --quiet google-cloud-pubsub

## Step 2 — Configure project and auth

In [ ]:
import os
PROJECT_ID = os.environ.get("PROJECT_ID", "your-gcp-project-id")  # <-- set your project
PUBSUB_TOPIC = "sla-email-events"

# If running in Workbench/Colab and not yet authenticated:
# !gcloud auth application-default login

print("PROJECT_ID:", PROJECT_ID)
print("PUBSUB_TOPIC:", PUBSUB_TOPIC)

## Step 3 — Load tickets and compute SLA flags (optional)

If your notebook already produces a DataFrame `df` with the required fields, skip this and ensure column names match the ones used below.

In [ ]:
import pandas as pd

# Adjust paths if needed
try:
    complaints = pd.read_csv("complaints_train.csv")
except FileNotFoundError:
    # Fallback for alternative path used earlier
    from pathlib import Path
    alt = Path("/mnt/data/Customer-Service/data/complaints_train.csv")
    complaints = pd.read_csv(alt) if alt.exists() else pd.DataFrame()

# Join customers to get email addresses (optional)
emails = None
for p in ["customers.csv", "/mnt/data/Customer-Service/data/customers.csv"]:
    try:
        emails = pd.read_csv(p)
        break
    except:
        pass

if emails is not None and not complaints.empty:
    df = complaints.merge(emails, on="customer_id", how="left")
else:
    df = complaints.copy()
    if "email" not in df.columns:
        df["email"] = "user@example.com"
    if "customer_name" not in df.columns:
        df["customer_name"] = "Customer"

SLA_MINUTES = {"low": 720, "medium": 240, "high": 120, "urgent": 60}

def estimate_first_response_minutes(row):
    base = {"low": 400, "medium": 200, "high": 100, "urgent": 50}.get(row.get("priority","medium"), 240)
    jitter = int(str(row.get("complaint_id", 0))[-2:]) % 80
    return base + jitter

if "first_response_minutes" not in df.columns and not df.empty:
    df["first_response_minutes"] = df.apply(estimate_first_response_minutes, axis=1)

if "sla_minutes" not in df.columns and not df.empty:
    df["sla_minutes"] = df["priority"].map(SLA_MINUTES)

if "sla_breached" not in df.columns and not df.empty:
    df["sla_breached"] = df["first_response_minutes"] > df["sla_minutes"]

expected_cols = ["complaint_id","email","customer_name","label","channel","priority","first_response_minutes","sla_minutes","sla_breached"]
print("Columns present:", df.columns.tolist())
missing = [c for c in expected_cols if c not in df.columns]
print("Missing (if any):", missing)
df.head() if not df.empty else "No data loaded" 

## Step 4 — Publisher helper

In [ ]:
from google.cloud import pubsub_v1
import json

publisher = pubsub_v1.PublisherClient()
topic_path = publisher.topic_path(PROJECT_ID, PUBSUB_TOPIC)

def publish_sla_event(row) -> str:
    payload = {
        "complaint_id": int(row["complaint_id"]),
        "email": row.get("email", "unknown@example.com"),
        "customer_name": row.get("customer_name", "Customer"),
        "label": row["label"],
        "channel": row["channel"],
        "priority": row["priority"],
        "first_response_minutes": int(row["first_response_minutes"]),
        "sla_minutes": int(row["sla_minutes"]),
        "sla_breached": bool(row["sla_breached"]),
    }
    data = json.dumps(payload).encode("utf-8")
    future = publisher.publish(topic_path, data=data)
    return future.result()  # Pub/Sub message ID

## Step 5 — Trigger email events

Choose **one** of the blocks below depending on what you want to send.

In [ ]:
# A) Publish for ALL rows
assert 'df' in globals() and not df.empty, "DataFrame df is empty or undefined."
count = 0
for _, r in df.iterrows():
    _ = publish_sla_event(r)
    count += 1
print(f"Published {count} SLA email events.")

In [ ]:
# B) Publish only breached (Apology emails)
assert 'df' in globals() and not df.empty, "DataFrame df is empty or undefined."
breached = df[df["sla_breached"] == True]
count = 0
for _, r in breached.iterrows():
    _ = publish_sla_event(r)
    count += 1
print(f"Published {count} apology-email events.")

In [ ]:
# C) Publish only met (Success emails)
assert 'df' in globals() and not df.empty, "DataFrame df is empty or undefined."
met = df[df["sla_breached"] == False]
count = 0
for _, r in met.iterrows():
    _ = publish_sla_event(r)
    count += 1
print(f"Published {count} success-email events.")

## Step 6 — Single-row test

In [ ]:
# Quick sanity check
assert 'df' in globals() and not df.empty, "DataFrame df is empty or undefined."
test_row = df.iloc[0]
print("Testing publish with:", test_row[["complaint_id","email","priority","sla_breached"]].to_dict())
print("Pub/Sub message id:", publish_sla_event(test_row))

## Step 7 — Verify and (optionally) schedule

**Verify:**
- Open **Cloud Logging** → filter by function name `send_sla_email` to confirm logs:
  - `Received SLA event: {...}`
  - `Email sent to ... (Apology|Success)`

**Schedule:**
- Schedule this notebook (Vertex Workbench Scheduler / Composer) or move the SLA compute into Cloud Run/Composer; publish to the **sla-email-events** topic on a cadence.

> To avoid duplicate sends, store last-sent status in BigQuery (or a CSV/DB) and publish only when status changes or first seen.